In [17]:
import numpy as np

from Resources.FDS.PeriodicTableFDS653 import PeriodicTable as PT
from Resources.ChemicalSpecies import ChemicalSpecies as CS

In [ ]:
# Define hydrogen atomic fraction in soot.
X_H = 0.1


# Define CO and soot yields.
y_CO = 0
y_Soot = 0.01

In [15]:
print(PT['C'], 
      PT['H'], 
      PT['O'], 
      PT['N'], 
      PT['Cl'])

['CARBON', 12.0107] ['HYDROGEN', 1.00794] ['OXYGEN', 15.9994] ['NITROGEN', 14.0067] ['CHLORINE', 35.453]


In [2]:
# Calculation of stoichiometry data for FDS input files. 
# Fuel is expected to be a list of the following structure: [C,H,O,N,CL]. 
# Elements in this list are coefficients (float) of the chemical formula.
# Fuel label is expected to be a string, will be used for the REAC_ID.
# Various yields can be specified, as well as the percentage of hydrogen 
# in the soot.
def stoichiometry(fuel,
                  fuel_label,
                  oxydizer_label,
                  product_label,
                  co2_yield=0.0,
                  co_yield=0.0,
                  soot_yield=0.0,
                  soot_h_fraction=0.0,
                  simple_air=False):
    
    # Calculate the molecular weight of the fuel molecule, in g/mol.
    w_fuel = fuel[0] * cMolWeight + \
             fuel[1] * hMolWeight + \
             fuel[2] * oMolWeight + \
             fuel[3] * nMolWeight + \
             fuel[4] * clMolWeight
    
    # Calculate the molecular weight of the soot, in g/mol.
    w_s = soot_h_fraction * hMolWeight + (1 - soot_h_fraction) * cMolWeight
    print('Soot weight:')
    print(w_s)
    
    # Calculate the stoichiometric coefficient of nitrogen; gas (products).
    # Nu means the greek letter.
    nu_n2 = fuel[3]/2
    print('Nu N2:')
    print(nu_n2)
    
    # Calculate the stoichiometric coefficient of soot; solid (products).
    # nu_s = w_fuel / w_s * soot_yield
    nu_s = stoichiometric_coefficient(w_fuel, w_s, soot_yield)
    print('Nu soot:')
    print(nu_s)
    
    # Calculate the stoichiometric coefficient of 
    # carbon monoxide; gas (products).
    # nu_co = w_fuel / co_mol_weight * co_yield
    nu_co = stoichiometric_coefficient(w_fuel, co_mol_weight, co_yield)
    print('Nu CO:')
    print(nu_co)
    
    # Calculate the stoichiometric coefficient of 
    # hydrogen chloride; gas (products).
    nu_hcl = fuel[4]
    # nu_hcl = stoichiometric_coefficient(w_fuel, clMolWeight, hclYield)
    print('Nu HCl:')
    print(nu_hcl)
    
    # Calculate the stoichiometric coefficient of water; gas (products).
    nu_h2o = fuel[1]/2 - soot_h_fraction / 2 * nu_s - fuel[4]
    print('Nu H2O:')
    print(nu_h2o)
    
    # Calculate the stoichiometric coefficient of 
    # carbon dioxide; gas (products).
    nu_co2 = fuel[0] - nu_co - (1 - soot_h_fraction) * nu_s
    print('Nu CO2:')
    print(nu_co2)
    
    # Calculate the stoichiometric coefficient of 
    # oxygen; gas (reactants).
    nu_o2 = nu_co2 + nu_co/2 + nu_h2o/2 - fuel[2]/2
    print('Nu O2:')
    print(nu_o2)

    # Volume fraction
    # Simple air follows the simplification that air consists out of 21 % 
    # of oxygen and 79 % of nitrogen.
    if simple_air is False:
        
        # The air_demand is based on the amount of oxygen needed for the
        # combustion. All the other components are scaled according to this 
        # value.
        air_demand = nu_o2 / airVolFrac[1]
        print('Air demand:')
        print(air_demand)
        print('nu_o2:')
        print(nu_o2)
        print('airVolFrac[1]:')
        print(airVolFrac[1])

        prod_n2 = air_demand * airVolFrac[0] + nu_n2
        print('Product N2:')
        print(prod_n2)
        
        prod_h2o = air_demand * airVolFrac[2] + nu_h2o
        print('product H2O:')
        print(prod_h2o)
        
        prod_co2 = air_demand * airVolFrac[3] + nu_co2
        print('Product CO2:')
        print(prod_co2)

    elif simple_air is True:
        air_demand = nu_o2 / airVolFracSimple[1]
        print('Air demand:')
        print(air_demand)

        prod_n2 = air_demand * airVolFracSimple[0] + nu_n2
        print('Product N2:')
        print(prod_n2)
        
        prod_h2o = nu_h2o
        print('product H2O:')
        print(prod_h2o)
        
        prod_co2 = nu_co2
        print('Product CO2:')
        print(prod_co2)
        pass
    else:
        print('Parameter "simple_air" needs to be True or False (bool)!')
        
    prod_hcl = nu_hcl
    print('Product HCl:')
    print(prod_hcl)
    
    prod_soot = nu_s
    print('Product soot:')
    print(prod_soot)
    
    prod_co = nu_co
    print('Product CO:')
    print(prod_co)
    
    # Calculate the total molar amount of all the products to normalise them.
    total_product_mol = (prod_n2 + prod_co2 + prod_h2o +
                         prod_hcl + prod_co + prod_soot)
    
    products = ['N2: '+str(prod_n2),
                'CO2: '+str(prod_co2),
                'H2O: '+str(prod_h2o),
                'HCl: '+str(prod_hcl),
                'CO: '+str(prod_co),
                'Soot: '+str(prod_soot)]
    
    # Normalise products, used for lumped species.
    products_normalised = [prod_n2/total_product_mol,
                           prod_co2/total_product_mol,
                           prod_h2o/total_product_mol,
                           prod_hcl/total_product_mol,
                           prod_co/total_product_mol,
                           prod_soot/total_product_mol]
    print(products_normalised)

    # return air_demand,products,total_product_mol,products_normalised
    
    product_species = ['NITROGEN',
                       'CARBON DIOXIDE',
                       'WATER VAPOR',
                       'HYDROGEN CHLORIDE',
                       'CARBON MONOXIDE',
                       'SOOT']
    
    species_invoke2, new_oxydizer = species_lump(oxydizer_label,
                                                 airComponents,
                                                 airVolFrac,
                                                 speciesInvoke)
    print("species_invoke2")
    print(species_invoke2)

    species_invoke3, new_product = species_lump(product_label,
                                                product_species,
                                                products_normalised,
                                                species_invoke2)
    print("species_invoke2")
    print(species_invoke2)
    
    print("species_invoke3")
    print(species_invoke3)

    reaction_para = [fuel_label,
                     oxydizer_label,
                     product_label,
                     1,
                     air_demand,
                     total_product_mol]
    new_reac = creat_reac_input(reaction_para[0], reaction_para)
    
    return species_invoke3, new_oxydizer, new_product, new_reac

In [ ]:
#    PRODUCTS
#    Molecular Weight (g/mol)            28.57721
#    Ambient Density (kg/m^3)            1.188
#    Initial Mass Fraction               0.000
#    Enthalpy of Formation (J/kg)     -2.84E+06

#    Sub Species                    Mass Fraction     Mole Fraction
#    NITROGEN                       7.016941E-01      7.158167E-01
#    CARBON DIOXIDE                 2.175212E-01      1.412456E-01
#    CARBON MONOXIDE                0.000000E+00      0.000000E+00
#    WATER VAPOR                    6.646607E-02      1.054335E-01
#    SOOT                           1.431862E-02      3.750416E-02

In [ ]:
#    AIR
#    Molecular Weight (g/mol)            28.76564
#    Ambient Density (kg/m^3)            1.196
#    Initial Mass Fraction               1.000
#    Enthalpy of Formation (J/kg)     -7.44E+04

#    Sub Species                    Mass Fraction     Mole Fraction
#    NITROGEN                       7.630774E-01      7.835682E-01
#    OXYGEN                         2.311814E-01      2.078229E-01
#    CARBON DIOXIDE                 5.919362E-04      3.869034E-04
#    WATER VAPOR                    5.149269E-03      8.222023E-03

In [ ]:
#    Fuel                              Heat of Combustion (kJ/kg)
#    TOLUENE                                           34963.6292

#    Stoichiometry

#    Primitive Species
#    Species ID                                 Stoich. Coeff.
#    TOLUENE                                            -1.000000
#    NITROGEN                                           -0.000000
#    OXYGEN                                             -7.609532
#    CARBON DIOXIDE                                      5.647113
#    WATER VAPOR                                         3.924840
#    SOOT                                                1.503208

#    Tracked (Lumped) Species
#    Species ID                                 Stoich. Coeff.
#    AIR                                            -36.615475
#    TOLUENE                                         -1.000000
#    PRODUCTS                                        40.081103

In [ ]:
#  Mass Fraction Transformation Matrix to Convert Species Mixtures 
# (Columns) to Primitive Species (Rows)

#                          AIR       TOLUENE   PRODUCTS  WATER VA
#    TOLUENE               0.000000  1.000000  0.000000  0.000000
#    NITROGEN              0.763077  0.000000  0.701694  0.000000
#    OXYGEN                0.231181  0.000000  0.000000  0.000000
#    CARBON DIOXIDE        0.000592  0.000000  0.217521  0.000000
#    CARBON MONOXIDE       0.000000  0.000000  0.000000  0.000000
#    WATER VAPOR           0.005149  0.000000  0.066466  1.000000
#    SOOT                  0.000000  0.000000  0.014319  0.000000

In [7]:
transform_matrix = np.array([[0.000000, 1.000000, 0.000000, 0.000000],
                             [0.763077, 0.000000, 0.701694, 0.000000],
                             [0.231181, 0.000000, 0.000000, 0.000000], 
                             [0.000592, 0.000000, 0.217521, 0.000000], 
                             [0.000000, 0.000000, 0.000000, 0.000000], 
                             [0.005149, 0.000000, 0.066466, 1.000000],
                             [0.000000, 0.000000, 0.014319, 0.000000]])

In [8]:
np.sum(transform_matrix, axis=0)

array([ 0.999999,  1.      ,  1.      ,  1.      ])